# 训练集数据预处理
*说明：数据预处理阶段，生成的特征包括（18个）：*

*出发机场、到达机场、航班编号、飞机编号、计划飞行时间、计划起飞时刻、航班月份、计划到达时刻、前序延误、起飞间隔、到达特情、出发特情、出发天气、出发气温、到达天气、到达气温、航空公司、航班性质。*

**目录**
1. 读取文件
2. 时间信息预处理
3. 前序航班的延误时间&到达与起飞间隔
4. 优化内存&类型转换
5. 特情
6. 天气
 - 气温
 - 天气情况
 - 将天气匹配到航班动态表中
7. 航班特征
8. 优化内存&类型转换
9. 将数据保存到文件中

# 读取文件
**文件对应关系为：**

1. flight.csv: 航班动态数据.csv
2. weather.csv: 城市天气.csv
3. airport.csv: 机场城市对应表.csv
4. spcial.csv: 特情.csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

# 获取当前的工作目录
pwd = os.getcwd()
# 将工作目录更改到训练集
os.chdir("原始训练集")
# ——————————————————————————读取数据—————————————————————————————— #
# 航班数据
flight_data = pd.read_table('flight.csv',sep=',',encoding='gb2312')
# 天气数据

weather = pd.read_csv('weather.csv')
# 城市与机场对应数据
airport_city = pd.read_excel('airport_city.xlsx')
# 特情
spcial = pd.read_excel('spcial.xlsx')
# 改回原来的工作目录
os.chdir(pwd)

## 读取后的航班动态展示

In [3]:
flight_data.head()

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常


# 时间信息预处理
## 生成的特征&字段
1. 计划飞行时间（单位：h）【可间接反映出航线距离】
2. 计划起飞日期&计划到达日期【用以匹配天气和特情信息】
3. 计划起飞时刻&计划到达时刻【不同时刻的航班】
4. 航班月份
5. 起飞延误时间（单位：h），取消的航班直接设置为延误10小时（大于3个小时即可）
6. 到达延误时间（单位：h）【主要用作后续航班的一个参考】

In [2]:
# 转化成日期格式
flight_data['计划起飞时间1'] = pd.to_datetime(flight_data['计划起飞时间'],unit='s',utc=True)
flight_data['计划到达时间1'] = pd.to_datetime(flight_data['计划到达时间'],unit='s',utc=True)
# 计划飞行时间
flight_data['计划飞行时间'] = flight_data['计划到达时间1'] - flight_data['计划起飞时间1']
flight_data['计划飞行时间'] = flight_data['计划飞行时间'].apply(lambda x: x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)

flight_data['计划飞行时间'] = flight_data['计划飞行时间']/3600  # 转换为小时
# 细分时间段
flight_data['计划起飞日期'] = flight_data['计划起飞时间1'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)
flight_data['计划起飞时刻'] = flight_data['计划起飞时间1'].apply(lambda x:x.strftime('%H') if not(pd.isnull(x)) else None)
flight_data['航班月份'] = flight_data['计划起飞时间1'].apply(lambda x:int(x.strftime('%m')) if not(pd.isnull(x)) else None)

flight_data['计划到达日期'] = flight_data['计划到达时间1'].apply(lambda x:x.strftime('%Y-%m-%d') if not(pd.isnull(x)) else None)
flight_data['计划到达时刻'] = flight_data['计划到达时间1'].apply(lambda x:x.strftime('%H') if not(pd.isnull(x)) else None)
# 延误
flight_data['起飞延误时间'] = pd.to_datetime(flight_data['实际起飞时间'],unit='s',utc=True) - pd.to_datetime(flight_data['计划起飞时间'],unit='s',utc=True)
flight_data['起飞延误时间'] = flight_data['起飞延误时间'].apply(lambda x: x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
flight_data['起飞延误时间'] = flight_data['起飞延误时间']/3600  # 转换为小时
flight_data['起飞延误时间'] = np.where(flight_data['航班是否取消'] == '取消',10,flight_data['起飞延误时间'])


flight_data['到达延误时间'] = pd.to_datetime(flight_data['实际到达时间'],unit='s',utc=True) - pd.to_datetime(flight_data['计划到达时间'],unit='s',utc=True)
flight_data['到达延误时间'] = flight_data['到达延误时间'].apply(lambda x: x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
flight_data['到达延误时间'] = flight_data['到达延误时间']/3600  # 转换为小时
flight_data['到达延误时间'] = np.where(flight_data['航班是否取消'] == '取消',10,flight_data['到达延误时间'])

# 删除后面用不着的字段，节约内存
del flight_data['计划起飞时间1']
del flight_data['计划到达时间1']
del(flight_data['实际起飞时间'])
del(flight_data['航班是否取消'])

# 前序航班的延误时间&到达与起飞间隔
这里前序航班的定义为：同一架飞机，当前航班的前一个航班即为当前航班的前序航班。比如，同一架飞机连续飞两个航班A：南京--北京，B：北京--西安，则A为B的前序航班。
## 延误时间（单位：h）
前序航班的延误时间定义为前序航班到达延误时间，即时间到达时间减去计划到达时间
## 到达与起飞间隔（单位：h）
当前航班的计划起飞时间与前序航班时间到达时间的间隔

In [ ]:
# 飞机编号有缺失值的统一编号为‘0’
flight_data['飞机编号']= flight_data['飞机编号'].fillna(0)
flight_data['前序延误'] = pd.Series()
flight_data['起飞间隔'] = pd.Series()
# 根据航班的飞机编号分组
grouped = flight_data.groupby(flight_data['飞机编号'])
chunks = []
for name,group in grouped:
#     print(name)
# 对每一架飞机按照计划起飞时间排序，以得到航班的顺序关系
    group = group.sort_values('计划起飞时间')
    a = pd.to_datetime(group['计划起飞时间'],unit='s',utc=True)[1:].reset_index(drop=True)
    b = pd.to_datetime(group['实际到达时间'],unit='s',utc=True)[0:len(group)-1].reset_index(drop=True)  
    group['起飞间隔'][1:] = a-b
    group['起飞间隔'] = group['起飞间隔'].apply(lambda x: x.days * 86400 + x.seconds if not(pd.isnull(x)) else None)
    group['起飞间隔'] = group['起飞间隔']/3600
#     前序航班的延误时间
    group['前序延误'][1:] = group['到达延误时间'][0:len(group)-1]
    chunks.append(group)
flight_data = pd.concat(chunks, ignore_index=True)
# 删除无用的变量和字段，释放内存
del(grouped)
del(group)
del(chunks)
del(flight_data['计划起飞时间'])
del(flight_data['计划到达时间'])
del(flight_data['实际到达时间'])
# 因为编号为0的飞机其实是缺失值，真实情况下的航班可能并没有顺序关系，所以暂时将其设置为NAN
flight_data['前序延误'][flight_data['飞机编号']==0] = np.NaN
flight_data['起飞间隔'][flight_data['飞机编号']==0] = np.NaN

# 优化内存&类型转换
**由于笔记本内存只有4G，所以时刻想着优化内存，以免死机**
也包括将部分字符型特征的转换为数值型特征


In [7]:
flight_data['出发机场'] = flight_data['出发机场'].astype('category')
flight_data['到达机场'] = flight_data['到达机场'].astype('category')
flight_data['航班编号'] = flight_data['航班编号'].astype('category')
flight_data['飞机编号'] = pd.to_numeric(flight_data['飞机编号'],errors='ignore',downcast='float')
flight_data['起飞延误时间'] = pd.to_numeric(flight_data['起飞延误时间'],errors='ignore',downcast='float')
flight_data['到达延误时间'] = pd.to_numeric(flight_data['到达延误时间'],errors='ignore',downcast='float')
flight_data['计划飞行时间'] = pd.to_numeric(flight_data['计划飞行时间'],errors='ignore',downcast='float')
flight_data['计划起飞时刻'] = pd.to_numeric(flight_data['计划起飞时刻'],errors='ignore',downcast='unsigned')
flight_data['计划到达时刻'] = pd.to_numeric(flight_data['计划到达时刻'],errors='ignore',downcast='unsigned')
flight_data['航班月份'] = pd.to_numeric(flight_data['航班月份'],errors='ignore',downcast='unsigned')
flight_data['计划起飞日期'] = flight_data['计划起飞日期'].astype('category')
flight_data['计划到达日期'] = flight_data['计划到达日期'].astype('category')
flight_data['前序延误'] = pd.to_numeric(flight_data['前序延误'],errors='ignore',downcast='float')
flight_data['起飞间隔'] = pd.to_numeric(flight_data['起飞间隔'],errors='ignore',downcast='float')

# 特情
特情这个特征的处理，没有区分特情的具体内容，只将特情发生的时间段对应到计划起飞和到达的时间，以0代表没有发生特情，1表示发生了特情，所以后面有继续优化这个特征的空间

In [9]:
del(spcial['收集时间'])
del(spcial['特情内容'])
# 将机场的代码转换为大写
for i in spcial.index:
    s = str(spcial['特情机场'][i])
    spcial['特情机场'][i] = s.upper()
#  提取出特情发生的开始结束日期和时刻
spcial['开始日期'] = spcial['开始时间'].apply(lambda x :str(x).split(' ')[0])
spcial['开始时刻'] = pd.Series()
spcial['结束时刻'] = pd.Series()
for i in spcial.index:
    if not(pd.isnull(spcial['开始时间'][i])):
        spcial['开始时刻'][i] = int(str(spcial['开始时间'][i]).split(' ')[1][:2])
    else :
        spcial['开始时刻'][i] = np.NaN
    if not(pd.isnull(spcial['结束时间'][i])):
        spcial['结束时刻'][i] = int(str(spcial['结束时间'][i]).split(' ')[1][:2])
    else :
        spcial['结束时刻'][i] = np.NaN
del(spcial['开始时间'])
del(spcial['结束时间'])
# 去重，避免左连接时发生多余样本
spcial = spcial.drop_duplicates(['特情机场','开始日期'])
# 到达城市特情
flight_data = pd.merge(flight_data,spcial,left_on=['到达机场','计划到达日期'],right_on=['特情机场','开始日期'],how='left',sort=False)
# 落在相应的时间段即为有特情
flight_data['到达特情'] = np.where((flight_data['计划到达时刻'] >=flight_data['开始时刻']) &
                               (flight_data['计划到达时刻']<= flight_data['结束时刻']),1,0)
del(flight_data['特情机场'])
del(flight_data['开始日期'] )
del(flight_data['开始时刻'])
del(flight_data['结束时刻'] )
# 出发城市特情
flight_data = pd.merge(flight_data,spcial,left_on=['出发机场','计划起飞日期'],right_on=['特情机场','开始日期'],how='left',sort=False)
# 落在相应的时间段即为有特情
flight_data['出发特情'] = np.where((flight_data['计划起飞时刻'] >=flight_data['开始时刻']) &
                               (flight_data['计划起飞时刻']<= flight_data['结束时刻']),1,0)
# 删除无用字段
del(flight_data['特情机场'])
del(flight_data['开始日期'] )
del(flight_data['开始时刻'])
del(flight_data['结束时刻'] )

# 天气
天气特征的提取主要包括气温特征和天气情况，其中：

1. 气温划分为3个取值，大于40度为高温，小于-10度为低温，其他为一般
2. 天气情况（小雨、阴天等）根据城市天气表，首先统计两年时间内所有天气情况在各地区总共出现的频率，出现频率小于50的天气情况统一划归为‘other’。

## 气温

In [33]:
# 气温处理
weather['气温'] = pd.Series()
weather['最高气温'][weather['最高气温']==' ']=  np.NaN
weather['最低气温'][weather['最低气温']==' ']=  np.NaN
weather['最高气温'] = weather['最高气温'].fillna('0')
weather['最高气温'] = weather['最高气温'].apply(lambda x : str(x).strip())
weather['最高气温'] = weather['最高气温'].apply(lambda x : str(x).split('.')[0])
weather['最高气温'] = weather['最高气温'].apply(lambda x : str(x).split(' ')[0])
weather['最低气温'] = weather['最低气温'].fillna('0')
weather['最低气温'] = weather['最低气温'].apply(lambda x : str(x).strip())
weather['最低气温'] = weather['最低气温'].apply(lambda x : str(x).split('.')[0])
weather['最低气温'] = weather['最低气温'].apply(lambda x : str(x).split(' ')[0])
weather['气温'] = weather['最高气温'].apply(lambda x: '高温' if int(x)>=40 else '一般')
weather['气温'] = np.where(weather['最低气温'].astype('int') < -10,'低温',weather['气温'])
del(weather['Unnamed: 5'])
del(weather['最高气温'])
del(weather['最低气温'])

## 天气情况
**注意：这里把频率小于50的天气设置为‘other’后，当前所有的天气情况需要保存下来，用以在测试集中天气情况的匹配**

In [11]:
weather = weather.drop_duplicates() # 只包含这三个字段
case = weather['天气'].value_counts()
case = case[case>=50]
weather_case = set(case.index)
weather['天气'] = weather['天气'].apply(lambda x: x if x in weather_case else 'other')
# 将机场编码对应到天气数据上面，根据城市名
airport_weather = pd.merge(weather,airport_city,left_on=['城市'],right_on=['城市名称'],how='left',sort=False)
del(airport_weather['城市名称'])
# 去除缺失值和重复的机场天气信息
airport_weather = airport_weather.dropna()
airport_weather = airport_weather.drop_duplicates(['日期','机场编码'])

## 将天气匹配到航班动态表中

In [12]:
# 出发城市
flight_data = pd.merge(flight_data,airport_weather,left_on=['出发机场','计划起飞日期'],right_on=['机场编码','日期'],how='left')
flight_data['出发天气'] = flight_data['天气']
flight_data['出发气温'] = flight_data['气温']
del(flight_data['天气'])
del(flight_data['机场编码'])
del(flight_data['城市'])
del[flight_data['日期']]
del(flight_data['气温'])
# 到达城市
flight_data = pd.merge(flight_data,airport_weather,left_on=['到达机场','计划到达日期'],right_on=['机场编码','日期'],how='left')
flight_data['到达天气'] = flight_data['天气']
flight_data['到达气温'] = flight_data['气温']
del(flight_data['天气'])
del(flight_data['机场编码'])
del(flight_data['城市'])
del[flight_data['日期']]
del(flight_data['气温'])
del(flight_data['计划起飞日期'])
del(flight_data['计划到达日期'])
del(flight_data['到达延误时间'])

# 航班特征
这里根据航班编号提取出的特征包括：

1. 航空公司
2. 航班的性质（摘自百度百科，一般尾号为字母的是补飞的，3位数字国内航班，4为数字国外航班）
 - 0：补飞
 - 1：国内正常
 - 2：国外

In [ ]:
flight_data['航空公司'] = flight_data['航班编号'].apply(lambda x : x[:2])
def f(x):
    if x[-1].isalpha():
        y = 0
    elif len(x[2:]) == 4:
        y = 1
    else:
        y = 2
    return(y)
flight_data['航班性质'] = flight_data['航班编号'].apply(f)

# 内存再优化

In [31]:
flight_data['出发天气'] = flight_data['出发天气'].astype('category')
flight_data['到达天气'] = flight_data['到达天气'].astype('category')
flight_data['出发气温'] = flight_data['出发气温'].astype('category')
flight_data['到达气温'] = flight_data['到达气温'].astype('category')
flight_data['航空公司'] = flight_data['航空公司'].astype('category')
flight_data['前序延误'] = pd.to_numeric(flight_data['前序延误'],errors='ignore',downcast='float')
flight_data['到达特情'] = pd.to_numeric(flight_data['到达特情'],errors='ignore',downcast='unsigned')
flight_data['出发特情'] = pd.to_numeric(flight_data['出发特情'],errors='ignore',downcast='unsigned')

In [30]:
flight_data.head()

,出发机场,到达机场,航班编号,飞机编号,计划飞行时间,计划起飞时刻,航班月份,计划到达时刻,起飞延误时间,前序延误,起飞间隔,到达特情,出发特情,出发天气,出发气温,到达天气,到达气温,航空公司,航班性质
0,YNZ,TYN,GJ8831,0.0,2.000000,1,1,3,10.0,NaN,NaN,0,0,多云,一般,晴,一般,GJ,1
1,PEK,HGH,FM9152,0.0,2.166667,5,1,7,10.0,NaN,NaN,0,0,雾,一般,多云,一般,FM,1
2,PEK,XIY,CA1205,0.0,2.250000,12,1,14,10.0,NaN,NaN,0,0,雾,一般,雾,一般,CA,1
3,CTU,TAO,CA4511,0.0,2.750000,22,1,1,10.0,NaN,NaN,0,0,晴,一般,晴转雾,一般,CA,1
4,PEK,CTU,CA4102,0.0,3.083333,2,1,5,10.0,NaN,NaN,0,0,雾,一般,晴转阴,一般,CA,1


# 将数据保存到文件中

In [38]:
# train_data1.csv和columns_types.pickle保存在"处理后训练集"文件夹下
# weather_case.csv 保存在原始测试集文件夹下
import pickle
os.chdir("处理后训练集")
flight_data.to_csv('train_data1.csv',index=False) # 保存训练数据
# 训练集数类型
colums_types = dict(flight_data.dtypes)
with open('colums_type.pickle', 'wb') as f:
     pickle.dump(colums_types, f) 
os.chdir(pwd)
os.chdir("原始测试集")
pd.Series(list(weather_case)).to_csv('weather_case.csv',index=False,header=True) # 保存天气情况数据
os.chdir(pwd)